## User parser agent Tests

In [ ]:
from agents.user_parser_agent import invoke_user_parser_agent

print("Running REMAS: user_parser_agent")
user_input = input("Type your message: ")

invoke_user_parser_agent(user_input=user_input)

Running REMAS: run_user_parser_agent
✅ Upserted 1 points into 'user_agent_listings' without resetting the collection.
✅ Uploaded user query with ID: 6e9fd5b8e158c408a80980e25e049dac to 'user_agent_listings'


'6e9fd5b8e158c408a80980e25e049dac'

In [3]:
from utils.qdrant_connection import count_qdrant
print("user Qdrant count:", count_qdrant("user_agent_listings"))

user Qdrant count: 1001


In [1]:
from utils.qdrant_connection import get_point_by_id
point_id = input("Enter the point ID to retrieve: ")
point = get_point_by_id("user_agent_listings", point_id)
if point:
    print(f"\n📌 Point ID: {point.id}")
    print("Payload:", point.payload)
else:
    print(f"No point found with ID: {point_id}")    
    


📌 Point ID: 6e9fd5b8-e158-c408-a809-80e25e049dac
Payload: {'listing_id': '6e9fd5b8e158c408a80980e25e049dac', 'state': ['Brooklyn', 'Queens'], 'picture_url': None, 'price': 2200, 'bedrooms': 2, 'available_from': 'October', 'soft_attributes': 'pet-friendly, close to public transportation, quiet neighborhood', 'source': 'user_query'}


## Owner parser agent Tests

In [1]:
from agents.owner_parser_agent import invoke_owner_parser_agent

print("Running REMAS: owner_parser_agent")
owner_input = input("Type your message: ")

invoke_owner_parser_agent(owner_input=owner_input)


Running REMAS: owner_parser_agent
✅ Upserted 1 points into 'owner_agent_listings' without resetting the collection.
✅ Uploaded owner listing with ID: 140d392e74131263a9ca770865306117 to 'owner_agent_listings'


'140d392e74131263a9ca770865306117'

In [2]:
from utils.qdrant_connection import count_qdrant
print("owner Qdrant count:", count_qdrant("owner_agent_listings"))

owner Qdrant count: 1008


In [4]:
from utils.qdrant_connection import get_point_by_id
point_id = input("Enter the point ID to retrieve: ")
point = get_point_by_id("owner_agent_listings", point_id)
if point:
    print(f"\n📌 Point ID: {point.id}")
    print("Payload:", point.payload)
else:
    print(f"No point found with ID: {point_id}")    


📌 Point ID: 140d392e-7413-1263-a9ca-770865306117
Payload: {'listing_id': '140d392e74131263a9ca770865306117', 'state': ['Astoria', 'Queens'], 'picture_url': 'https://example.com/apt-photos/astoria2.jpg', 'price': 2400, 'bedrooms': 2, 'available_from': 'October', 'soft_attributes': 'spacious, 5 minutes from the subway, balcony with city views, newly renovated kitchen, in-unit washer/dryer'}


In [10]:
# pront count similatity 
from utils.qdrant_connection import count_qdrant
print("similarity_collection Qdrant count:", count_qdrant("similarity_collection"))     

similarity_collection Qdrant count: 30064


In [11]:
from utils.qdrant_connection import scroll_qdrant
print("Scrolling through similarity_collection:")
list = scroll_qdrant("similarity_collection", limit=10)[0]
for point in list:
    print(f"\n📌 Point : {point}")

Scrolling through similarity_collection:

📌 Point : id='00002dfd-2619-8928-90fc-a885f9bf7e7c' payload={'user_id': 'b4c4af9a-95cf-3b26-d0d3-6cb6285752e8', 'owner_id': 'd9e08424-bc77-656e-72f3-44c488406268', 'score': 0.55240315, 'filter_used': {'collection': 'user_agent_listings', 'state': ['New York City'], 'price_owner': None, 'bedrooms_owner': 5, 'available_from': None}, 'timestamp': 1754906840.3216486} vector=None shard_key=None order_value=None

📌 Point : id='00018f3a-6b1b-7701-9322-70b25d96fae7' payload={'user_id': '3e91e308-97c8-5c72-b42a-b61498983051', 'owner_id': '5fe3c73d-69fa-29d4-8b1b-34f6fab0f107', 'score': 0.6045309, 'filter_used': {'collection': 'user_agent_listings', 'state': ['Crown Heights', 'Brooklyn', 'New York City'], 'price_owner': 2000, 'bedrooms_owner': 1, 'available_from': None}, 'timestamp': 1754906437.3464315} vector=None shard_key=None order_value=None

📌 Point : id='00034545-0053-1cfc-2e2e-670af297d33d' payload={'user_id': '07a040bc-f51e-501d-6d89-9ed5dbf4d75

## matching

In [1]:

from agents.owner_parser_agent import invoke_owner_parser_agent
from agents.user_parser_agent import invoke_user_parser_agent
from agents.matching_agent import match_for_new_user, match_for_new_owner

# --- 1) Inputs (aligned) ---
owner_input = """
Spacious 2-bedroom apartment in san fransisco, just 5 minutes from the subway.
Includes a balcony with city views, newly renovated kitchen, and in-unit washer/dryer.
$2400 per month, available starting October.
Picture: https://example.com/apt-photos/astoria2.jpg
""".strip()

user_input = """
Looking for a 2-bedroom place in san fransisco, ideally close to the subway.
Quiet building, good for working from home; in-unit laundry is a plus.
My budget is up to $2,500 per month and I'd like to move in by October.
""".strip()

# --- 2) Parse + upload (each returns the new point id) ---
print("➕ Uploading owner listing…")
owner_point_id = invoke_owner_parser_agent(owner_input)
print("Owner point id:", owner_point_id)

print("\n➕ Uploading user query…")
user_point_id = invoke_user_parser_agent(user_input)
print("User point id:", user_point_id)

➕ Uploading owner listing…
✅ Upserted 1 points into 'owner_agent_listings' without resetting the collection.
✅ Uploaded owner listing with ID: ec4161ac90d8bb2a867fe5fe03a441b7 to 'owner_agent_listings'
Owner point id: ec4161ac90d8bb2a867fe5fe03a441b7

➕ Uploading user query…
✅ Upserted 1 points into 'user_agent_listings' without resetting the collection.
✅ Uploaded user query with ID: d2440de8efa334ada83f6a857954a293 to 'user_agent_listings'
User point id: d2440de8efa334ada83f6a857954a293


In [2]:
# --- 3) Match both directions ---
TOP_K = 10

print("\n🔎 Matching for NEW USER → owner listings …")
user_matches = match_for_new_user(user_point_id, top_k=TOP_K)

print("🔎 Matching for NEW OWNER → user queries …")
owner_matches = match_for_new_owner(owner_point_id, top_k=TOP_K)


🔎 Matching for NEW USER → owner listings …
🔎 Matching for NEW OWNER → user queries …


In [ ]:
from utils.qdrant_connection import print_owner_matches_with_details , print_user_matches_with_details

print_user_matches_with_details(user_matches, top_k=5)
print_owner_matches_with_details(owner_matches, top_k=5)


Top 2 owners for this user:
  score=0.6803  owner_id=ec4161ac-90d8-bb2a-867f-e5fe03a441b7
    [state: San Francisco | price: 2400 | bedrooms: 2 | available_from: October] soft: spacious, 5 minutes from the subway, balcony with city views, newly renovated kitchen, in-unit washer/dryer
  score=0.5245  owner_id=d3dc4afb-25c2-f237-5b25-32620bcd2c94
    [state: San Francisco | price: 1600 | bedrooms: 2 | available_from: October] soft: centrally located, stunning views of the Bay Bridge, in-unit laundry, private balcony, outdoor swimming pool, hot tub, g…
Top 4 users for this owner:
  score=0.6803  user_id=d2440de8-efa3-34ad-a83f-6a857954a293
    [state: San Francisco | price: 2500 | bedrooms: 2 | available_from: October] soft: quiet building, good for working from home, close to the subway, in-unit laundry is a plus
  score=0.5047  user_id=287d155c-66bb-e4f9-5e90-3cca55623a6d
    [state: San Francisco | price: 2500 | bedrooms: 1 | available_from: October] soft: modern, centrally located, ea

## Decicion

In [1]:
# Test daily_llm_showing_decisions on the sampled collection

import csv
import time
from datetime import datetime

from agents.manage_showings_agent import daily_llm_showing_decisions, OWNER_COLLECTION
from utils.qdrant_connection import client  # for count

# ------------ Config you can tweak ------------
TOP_K = 10             # how many matches per owner to consider
WRITE_CSV = True       # set False to skip writing a log file
MAX_PREVIEW = 8        # how many rows to print in console
# ----------------------------------------------

# Quick sanity check: sampled collection exists & size
try:
    cnt = client.count(collection_name=OWNER_COLLECTION, exact=True).count
    print(f"Testing collection: '{OWNER_COLLECTION}' | count = {cnt}")
except Exception as e:
    raise RuntimeError(f"Could not count collection '{OWNER_COLLECTION}': {e}")

t0 = time.time()
results = daily_llm_showing_decisions(top_k=TOP_K)
dt = time.time() - t0

# Aggregate stats
n_total = len(results)
n_errors = sum(1 for r in results if "error" in r)
n_decided = sum(1 for r in results if "decision" in r)
n_show = sum(1 for r in results if r.get("decision", {}).get("show") == "1")
total_invites = sum(int(r["decision"]["num"]) for r in results if r.get("decision", {}).get("show") == "1")

print(f"\nRun time: {dt:.2f}s")
print(f"Owners processed: {n_total} | with decisions: {n_decided} | errors: {n_errors}")
print(f"SHOW=1: {n_show} | total invites: {total_invites}")

# Preview a few rows
print("\nSample decisions:")
preview = 0
for r in results:
    if "error" in r:
        print(f"- owner_id={r['owner_id']}  ERROR: {r['error_type'] if 'error_type' in r else ''} {r['error']}")
    else:
        dec = r["decision"]
        print(f"- owner_id={r['owner_id']}  considered={r['considered']}  mean_top5={r['mean_top5']}  decision={dec}  sample={r['sample']}")
    preview += 1
    if preview >= MAX_PREVIEW:
        break

# Optional: write CSV log
if WRITE_CSV and results:
    ts = datetime.utcnow().strftime("%Y%m%d-%H%M%S")
    path = f"logs/showing_decisions_run_{OWNER_COLLECTION}_{ts}.csv"
    with open(path, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(["owner_id", "considered", "mean_top5", "show", "num", "sample", "error_type", "error"])
        for r in results:
            if "error" in r:
                w.writerow([r.get("owner_id"), "", "", "", "", "", r.get("error_type",""), r.get("error","")])
            else:
                dec = r["decision"]
                w.writerow([
                    r["owner_id"],
                    r["considered"],
                    r["mean_top5"],
                    dec.get("show"),
                    dec.get("num"),
                    r.get("sample"),
                    "",
                    ""
                ])
    print(f"\nSaved CSV → {path}")


Testing collection: 'sampled_owner_agent_listings' | count = 50


ManageShowings:   0%|          | 0/50 [00:00<?, ?owner/s]


Run time: 65.26s
Owners processed: 50 | with decisions: 50 | errors: 0
SHOW=1: 41 | total invites: 175

Sample decisions:
- owner_id=044e9ae9-c6a7-c53d-a570-2a26b4e455ed  considered=10  mean_top5=0.496  decision={'show': '1', 'num': '3'}  sample=[{'user_id': '698a8e67-0083-b180-80fd-64514771d158', 'score': 0.7215}, {'user_id': 'b5579128-f5d0-6056-747e-75e0ad260a8c', 'score': 0.4769}, {'user_id': 'c6b8a2b9-c691-2780-8711-d14e3b80241a', 'score': 0.4531}]
- owner_id=04cc4a5f-1b97-b3e0-be73-94dc559d189b  considered=6  mean_top5=0.5221  decision={'show': '1', 'num': '3'}  sample=[{'user_id': 'b222c70e-08ba-0212-fc0d-7d6b45a5b64b', 'score': 0.7772}, {'user_id': '04315916-c790-51ad-2d6e-20a4a380a2d7', 'score': 0.5228}, {'user_id': 'cdafe5f3-afad-0463-cba3-e97ac54abefc', 'score': 0.4898}]
- owner_id=06e4884b-aa37-22a6-e9bd-1fd6a76c3f8a  considered=10  mean_top5=0.6179  decision={'show': '1', 'num': '5'}  sample=[{'user_id': '4e959be2-f2d2-d685-5a46-430b21f93d6b', 'score': 0.637}, {'user_id': 